<a href="https://colab.research.google.com/github/Samantha996/Machine-Learning/blob/main/Python_StreamlitApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Streamlit App

This web app displays the 3 most common Machine Learning algorithmns (KNN, SVM, Randon Forest) and how accurate they are with three different datasets.

In [ ]:
#installing packages
!pip install streamlit #can use streamlit-demo to see different animations
!pip install matplotlib
!pip install scikit-learn
!pip install pyngrok==4.1.1 #needed to create streamlit app through colaboratory
!pip install numpy

     |████████████████████████████████| 7.5MB 5.5MB/s 
     |████████████████████████████████| 112kB 34.7MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 163kB 49.3MB/s 
     |████████████████████████████████| 4.6MB 53.1MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 122kB 53.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=41db5b4c68ce6b3f6d16ad7b2b1b1811c23a0870af0e3056c5c41549439fe689
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=dc6328660fe5052325d6469b9776990e65adc9a803a8091599cfafad85ed1a22
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


Below is our actual web app code that will be hosted onto the streamlit app using ngrok.

In [ ]:
%%writefile app.py
#importing packages
import streamlit as st
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA #principal component algo
import numpy as np

st.title("Choosing the Best Classifier")
dataset_name = st.sidebar.selectbox("Select Dataset", ("Iris", "Breast Cancer", "Wine"))

classifier_name = st.sidebar.selectbox("Select Classifier", ("KNN", "SVM", " Random Forest"))

#datasets are already included in python packages
def get_dataset(dataset_name):
  if dataset_name == "Iris":
    data = datasets.load_iris()
  elif dataset_name == "Breast Cancer":
    data = datasets.load_breast_cancer()
  else:
    data = datasets.load_wine()
  X = data.data
  y = data.target 
  return X, y 

X, y = get_dataset(dataset_name)
st.write("Shape of the Dataset", X.shape)
st.write("Number of different classes", len(np.unique(y)))

def add_parameter_ui(clf_name):
  params = dict()
  if clf_name == "KNN":
    K = st.sidebar.slider("K", 1, 15)
    params["K"] = K 
  elif clf_name == "SVM":
    C = st.sidebar.slider("C", 0.01, 10.0)
    params["C"] = C 
  else:
    max_depth = st.sidebar.slider("max_depth", 2, 15)
    n_estimators = st.sidebar.slider("n_estimators", 1, 100)
    params["max_depth"] = max_depth
    params["n_estimators"] = n_estimators
  return params

params = add_parameter_ui(classifier_name)

def get_classifier(clf_name, params):
  if clf_name == "KNN":
    clf = KNeighborsClassifier(n_neighbors=params["K"])
  elif clf_name == "SVM":
    clf = SVC(C = params["C"]) 
  else:
      clf = RandomForestClassifier(n_estimators=params['n_estimators'],
      max_depth=params['max_depth'], random_state = 1234)
  return clf
   
clf = get_classifier(classifier_name, params)

# Classification

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

#Accuracy Score
acc = accuracy_score(y_test, y_pred)
st.write(f"Classifier = {classifier_name}")
st.write(f"Accuracy = {acc}")

#Plot Data
pca = PCA(2) #dim
X_projected = pca.fit_transform(X)

x1 = X_projected[:, 0]
x2 = X_projected[:, 1]

fig = plt.figure()
plt.scatter(x1, x2, c=y, alpha=0.8, cmap="viridis")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar()


st.pyplot(fig)



Overwriting app.py


Here we are importing ngrok in order to create a localhost for this web app.

In [ ]:
#needed to create local host
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

http://090ffcf225a5.ngrok.io


In [ ]:
Credit: Python Engineer (https://www.youtube.com/watch?v=Klqn--Mu2pE)